# Setup

In [2]:
import psycopg as pg
import pandas as pd

Example changing the amount of bricks in external walls

In [4]:
SEF = 1.2 

In [8]:
FloorHeight=3.5

In [12]:
WindowWallRatio=0.2

In [5]:
get_perimeter_sql=f"SELECT COALESCE(SQRT(b.total_building_area/b.n_floors),SQRT(b.built_area))*2*(%s+1/%s) as perimeter" % (SEF,SEF) #Rough approximation if the building has storeys of different sizes

In [14]:
SQL="""
WITH brick_ext_walls_new AS
(SELECT 
    b.bbr_id bbrid,
    typ.id,
    typ.name,
    perimeter,
    ext_wall_surf,
    brick_thickness,
    pr.name product,
    amount_product,
    pmap.unit unit
FROM buildings b
INNER JOIN buildings_to_ext_walls bmap
    ON b.bbr_id = bmap.bbr_id
INNER JOIN ext_wall_types typ
    ON typ.id=2
INNER JOIN ext_walls_to_subcomponents submap
    ON typ.id = submap.ext_wall_id
INNER JOIN subcomponents sc
    ON sc.lcabyg_id = submap.subcomponent_id
INNER JOIN subcomponents_to_products pmap
    ON pmap.subcomponent_id = sc.lcabyg_id
INNER JOIN products pr
    ON pr.lcabyg_id = pmap.product_id,
LATERAL (%s) ltp,
LATERAL (SELECT (CASE WHEN b.n_floors IS NOT NULL THEN perimeter*b.n_floors*%s*(1-%s) ELSE perimeter*%s*(1-%s) END) AS ext_wall_surf) lt1,
LATERAL (SELECT (CASE WHEN b.n_floors IS NOT NULL THEN 0.228 * (0.85+(b.n_floors+1)/8) ELSE 0.228 END) AS brick_thickness) lt2,
LATERAL (SELECT ext_wall_surf*brick_thickness*pmap.amount/0.336 AS amount_product) lt3
WHERE pr.name NOT LIKE '%%plaster%%'
)

UPDATE results_material_amounts
SET amount = brick_ext_walls_new.amount_product
FROM brick_ext_walls_new
WHERE (results_material_amounts.bbr_id, results_material_amounts.element, results_material_amounts.product)=(brick_ext_walls_new.bbrid, 'ext_wall', brick_ext_walls_new.product) 

""" %(get_perimeter_sql, FloorHeight, WindowWallRatio, FloorHeight, WindowWallRatio)

Then we can always update the total amounts table:

In [15]:
update_tot_amounts_sql="""
WITH t AS(
SELECT 
    element,
    SUM(amount) sum,
    unit,
    product
FROM results_material_amounts
GROUP BY (element, unit, product))

UPDATE tot_material_amounts tma
SET amount=t.sum
FROM t
WHERE (tma.element, tma.unit, tma.product) = (t.element, t.unit, t.product)
"""

We can fix foundations similarly with the 60 buildings study: material amount seems proportional to floor area, not footprint.